In [1]:
import os
import copy
import random
import gym
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

from itertools import permutations
from sklearn.model_selection import KFold, GridSearchCV

from multiprocessing import set_start_method
import multiprocessing as mp

path = os.path.abspath('..')
if path not in sys.path:
    sys.path.append(path)

from seal.agents.default_config import DEFAULT_CONFIG as config
from seal.agents.dqn import DQNAgent
# from seal.agents.qr_dqn import QuantileAgent
# from seal.agents.multi_head_dqn import MultiHeadDQNAgent
# from seal.agents.discrete_bcq import DiscreteBCQAgent

from seal.algos.kfold import CVS, KFoldCV
from seal.algos.advantage_learner import AdvantageLearner
from seal.algos.behavior_cloning import BehaviorCloning
from seal.algos.density_ratio import VisitationRatioModel
from seal.algos.fqe import FQE

def one_step(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    path = './data/mh/ddqn/trajs_mh.pkl'
    nfolds = 5
    n_splits = 5
    ckpts = (np.arange(10) + 1)*1000
    
    num_actions = 5
    # configures
    config['online'] = False
    config['lr'] = 5e-4
    config['decay_steps'] = 50000
    config['max_training_steps'] = 10000
    config['training_steps_to_checkpoint'] = 1000
    config['training_steps_to_eval'] = 100000
    config['hiddens'] = [64,64]
    config['double'] = True
    config['dueling'] = False

    index = pd.MultiIndex.from_product([np.arange(nfolds), ckpts])
    columns = ['dqn',  'seal']
    rets = pd.DataFrame(index=index, columns=columns)

    print('-'*20, 'start', '-'*20)
    cvs = CVS(path, n_splits=nfolds, random_state=seed)
    cvs.split()
    for fold in range(nfolds):
        train_path = cvs.train_paths[fold] + 'trajs.pkl'
        kf = KFoldCV(train_path, n_trajs=None, n_splits=n_splits, shuffle=False, random_state=seed)
        kf.split()

        print('-'*20, 'training agent', '-'*20)
        # agent
        config['persistent_directory'] = kf.agent_path
        config['checkpoint_path'] = kf.ckpt_path
        agent = DQNAgent(num_actions=num_actions, config=config)
        agent.learn()

        print('-'*20, 'training agents', '-'*20)
        # agent_1, ..., agent_K
        for idx in range(kf.n_splits):
            config_idx = copy.deepcopy(config)
            config_idx['persistent_directory'] = kf.agent_paths[idx]
            config_idx['checkpoint_path'] = kf.ckpt_paths[idx]
            agent_idx = DQNAgent(num_actions=num_actions, config=config_idx)
            agent_idx.learn()

        # fitted q evaluation
        test_path = cvs.test_paths[fold] + 'trajs.pkl'
        with open(test_path, 'rb') as f:
            trajs = pickle.load(f)

        print('-'*20, 'behavior cloning', '-'*20)
        # behavior cloning
        bc = BehaviorCloning(num_actions=num_actions)
        states  = np.array([transition[0] for traj in kf.trajs for transition in traj])
        actions = np.array([transition[1] for traj in kf.trajs for transition in traj])
        bc.train(states, actions)

        for ckpt in ckpts:
            print('-'*20, 'ckpt: ', ckpt, '-'*20)
            agent = DQNAgent(num_actions=num_actions, config=config)
            agent.load(kf.ckpt_path + 'offline_ddqn_{}.ckpt'.format(ckpt))

            agents = []
            for idx in range(kf.n_splits):
                config_idx = copy.deepcopy(config)
                config_idx['persistent_directory'] = kf.agent_paths[idx]
                config_idx['checkpoint_path'] = kf.ckpt_paths[idx]
                agent_idx = DQNAgent(num_actions=num_actions, config=config_idx)
                agent_idx.load(kf.ckpt_paths[idx] + 'offline_ddqn_{}.ckpt'.format(ckpt))
                agents.append(agent_idx)
            states, qvalues, qtildes = kf.update_q(agents, bc)

            print('-'*20, 'adv learner', '-'*20)
#             advs1 = qvalues - qvalues.mean(axis=1, keepdims=True)
#             agent1 = AdvantageLearner(num_actions=num_actions)
#             agent1._train(states, advs1)
            
            advs2 = qtildes - qtildes.mean(axis=1, keepdims=True)
            agent2 = AdvantageLearner(num_actions=num_actions)
            agent2._train(states, advs2)

            print('-'*20, 'fqe on dqn & seal', '-'*20)
            fqe_dqn = FQE(agent.greedy_actions, num_actions=num_actions, activation='tanh', hiddens=config['hiddens'], max_iter=100, eps=0.0015)
            fqe_dqn.train(trajs)
#             fqe_dml = FQE(agent1.greedy_actions, num_actions=num_actions)
#             fqe_dml.train(trajs)
            fqe_seal = FQE(agent2.greedy_actions, num_actions=num_actions, activation='tanh', hiddens=config['hiddens'], max_iter=100, eps=0.0015)
            fqe_seal.train(trajs)

            rets.loc[(fold, ckpt), 'dqn'] = fqe_dqn.values
#             rets.loc[(fold, ckpt), 'dml'] = fqe_dml.values
            rets.loc[(fold, ckpt), 'seal'] = fqe_seal.values
            
    return rets

In [ ]:
save_path = './data/mh/ddqn/'
pool = mp.Pool(5)
rets = pool.map(one_step, range(5))
pool.close()

with open(save_path + 'rets_ddqn_mh.pkl', 'wb') as f:
    pickle.dump(rets, f)

---------------------------------------- start--------------------   start --------------------start
----------------------------------------  start-------------------- start--------------------
 --------------------
 --------------------

-------------------- training agent --------------------
-------------------- training agent --------------------
-------------------- training agent --------------------
--------------------Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/218175338/fold0/train/agent/trajs.pkl!
Refresh buffer every 1000000 sampling! 
training agent --------------------

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you 



saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/218175338/fold0/train/agent/ckpt/offline_ddqn_1000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/932136058/fold0/train/agent/ckpt/offline_ddqn_1000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/717354021/fold0/train/agent/ckpt/offline_ddqn_1000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/209652396/fold0/train/agent/ckpt/offline_ddqn_1000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/798842024/fold0/train/agent/ckpt/offline_ddqn_1000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/717354021/fold0/train/agent/ckpt/offline_ddqn_2000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/218175338/fold0/train/agent/ckpt/offline_ddqn_2000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/932136058/fold0/train/agent/ckpt/offline_ddqn_2000.ckpt
saving model weights a



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/932136058/fold0/train/agent0/trajs0.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the


saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/798842024/fold0/train/agent0/ckpt/offline_ddqn_9000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/717354021/fold0/train/agent0/ckpt/offline_ddqn_10000.ckpt
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/717354021/fold0/train/agent1/trajs1.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this w


saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/798842024/fold0/train/agent1/ckpt/offline_ddqn_10000.ckpt
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/798842024/fold0/train/agent2/trajs2.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dty

...

30 WARNING:tensorflow:A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.target diff: 
0.0023479725670707313 WARNING:tensorflow:Unresolved object in checkpoint: (root).dense_layers.0.biasvalues: 
-----WARNING:tensorflow:Unresolved object in checkpoint: (root).dense_layers.2.kernel 


Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/932136058/fold1/train/agent0/trajs0.pkl!
Refresh buffer every 1000000 sampling!
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/ddqn/tmp/932136058/fold1/train/agent1/trajs1.pkl!
Refresh buffer every 1000000 sampling!
Loaded trajectories fr


-----iteration:  31 target diff:  0.002694206751995486 values:  -60.837803 ----- 

-------------------- adv learner --------------------
-----iteration:  6 target diff:  0.0019366479790635998 values:  -59.292416 ----- 

-----iteration:  32 target diff:  0.0030169597126491086 values:  -60.6468 ----- 

-------------------- fqe on dqn & sale --------------------
-----iteration:  33 target diff:  0.00307002779152313 values:  -60.46478 ----- 

-----iteration:  7 target diff:  0.0018777471759458687 values:  -59.258327 ----- 

-----iteration:  34 target diff:  0.0023668461696867253 values:  -60.262478 ----- 

-----iteration:  8 target diff:  0.0019040422180037259 values:  -59.35152 ----- 


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change ju


-------------------- adv learner --------------------
-----iteration:  49 target diff:  0.003661578579232641 values:  -58.18912 ----- 


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If

-----iteration:  0 -----iteration: target diff:  63  0.9182128450378767 target diff: values:  0.0029234067478114594 -58.780193  values: -----  -56.566765
 
----- 

-----iteration:  1 target diff:  0.002090775153326039 values:  -58.862984 ----- 

-----iteration:  64 target diff:  0.0030972271729244327 values:  -56.41961 ----- 

-----iteration:  8 target diff:  0.0020053336274588702 values:  -49.416008 ----- 

-----iteration:  7 target diff:  0.00152639022261745 values:  -55.535786 ----- 

-----iteration:  2 target diff:  0.002278283206721646 values:  -58.888985 ----- 

-----iteration:  65 target diff:  0.0028244735658544524 values:  -56.26322 -----iteration:  -----iteration:  8-----  9
target diff: 
  0.001502738288772013target diff:   values: 0.0015776371026515773  -55.51667values:   ----- -49.407803
 
----- 

-------------------- fqe on dqn & sale --------------------
 target diff:  0.0021050805045591857 values:  -58.97578 ----- 

-----iteration:  66 target diff:  0.003215011052274545

-----iteration:  77 target diff:  0.002865923884455071 values:  -54.659565 ----- 

-----iteration:  17 target diff:  0.0019832288623196238 values:  -49.67808 ----- 

-----iteration: -----iteration:   04  target diff: target diff:  0.0019353511826437268  values:  0.9171623351918939 values: -60.374836  ----- -59.696514
-----iteration:   
78-----  
target diff: 
 0.003439121477723968 values:  -54.600655 ----- 

-----iteration:  18 target diff:  0.0019009968505247718 values:  -49.617657 ----- 

-----iteration:  -----iteration:  179 target diff:   0.0035220819358847233target diff:   values: 0.00261874771448351 -59.79064  values:  ------54.523502 
 
----- 

-----iteration:  5 target diff:  0.0015434684876042162 values:  -60.355366 ----- 

-----iteration:  80 target diff:  0.002254511211720067 values:  -54.374603 ----- 

-----iteration:  19 target diff:  0.0018059051158282376 values:  -49.55631 ----- 

-----iteration:  2 target diff:  0.002982955131520575 values:  -59.869698 ----- 

-----iter